# Exercise 2 - shell, pipes, and csvkit

In this exercise, we'll review a few shell commands and explore working with pipes and csvkit.

You will need to fill in commands whenever prompted.  Please replace the text with your solution.

Remember to submit your completed `.ipynb` file to Blackboard and to add/commit it to your Git repository and push it to GitHub.


## Part 1 - shell commands, redirection, and pipes

### Basic shell commands and redirection

Create a directory called `part1` using `mkdir`.

In [1]:
!mkdir part1

Rename `part1` to `partone` using `mv`.

In [2]:
!mv part1 partone
!ls

exercise-02.ipynb  partone


Create a file named `filelist.txt` using the output from `ls` and the output redirector `>`.

In [3]:
!ls > filelist.txt

In [4]:
!cat filelist.txt

exercise-02.ipynb
filelist.txt
partone


Append to `filelist.txt` using the output appending redirector `>>`.  Note the difference between the single `>` and double `>>`.

In [5]:
!ls >> filelist.txt
!cat filelist.txt

exercise-02.ipynb
filelist.txt
partone
exercise-02.ipynb
filelist.txt
partone


In [6]:
!ls > filelist.txt
!cat filelist.txt

exercise-02.ipynb
filelist.txt
partone


What's the difference between `>` and `>>`?


In [12]:
# Command > is used to write to a file and >> is used to append to a file. 
# In above cases, > was used to write the output of ls to filelist.txt and when the file was already present, its contents were overwritten.
# When you use >>, it appended the new contents after the old one, if the file was already present.

### Your turn

Complete the following tasks in the cells provided.  All the tests in the testing cells (with the `assert` statements) below should pass without error - be sure to execute those as well, and if you see errors, fix your answer and try testing again until there are no errors.

Create a directory called `mydirectory`.

In [13]:
!mkdir mydirectory

In [14]:
import os
assert 'mydirectory' in os.listdir('.')

Using `ls` and output redirection, create a file called `myfiles.txt` in the directory `mydirectory` that contains the list of files in the current directory.

In [15]:
!ls > mydirectory/myfiles.txt

In [16]:
assert 'myfiles.txt' in os.listdir('mydirectory')

In [17]:
myfiles = open('mydirectory/myfiles.txt').read()
assert 'exercise-02.ipynb' in myfiles

Clean up the directory you just created by removing its contents (the file you created) using `rm`.

In [18]:
!rm mydirectory/*

In [19]:
assert 'myfiles.txt' not in os.listdir('mydirectory')

Now remove the directory itself using `rmdir`.

In [20]:
!rmdir mydirectory

In [21]:
assert 'mydirectory' not in os.listdir('.')

### Filters and pipes

Let's look at something a little more interesting.  Download the text of Herman Hesse's *Siddhartha* from [Project Gutenberg](http://www.gutenberg.org/):

In [22]:
!wget https://www.gutenberg.org/cache/epub/2500/pg2500.txt

--2016-09-09 09:45:15--  https://www.gutenberg.org/cache/epub/2500/pg2500.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 241176 (236K) [text/plain]
Saving to: ‘pg2500.txt’

pg2500.txt          100%[===================>] 235.52K   833KB/s    in 0.3s    

2016-09-09 09:45:16 (833 KB/s) - ‘pg2500.txt’ saved [241176/241176]



*Note*: sometimes Project Gutenberg restricts access.  If that creates an error for you, you should be able to `wget` the same file from our class repository on GitHub at the url https://github.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/raw/master/exercises/pg2500.txt.

However you get the file, let's rename it to something easier to remember.

In [23]:
!mv pg2500.txt siddhartha.txt

`head` and `tail` are very useful.  They let you take a quick peek at the start and end of files.

In [24]:
!head siddhartha.txt

In [25]:
!tail siddhartha.txt

`grep` is one of the most useful filters.  It lets you search for and match lines that contain specific expressions.  For example, to find mentions of "copyright":

In [26]:
!grep copyright siddhartha.txt

Notice anything that those lines have in common?

Let's add a little more information by including the `-n` flag to add matching line numbers.

In [27]:
!grep -n copyright siddhartha.txt

Now let's look for any mention of "river".  This will match a lot of text, so we'll just take the first 10 matching lines by *piping* the output from `grep` into `head`.

In [28]:
!grep -n river siddhartha.txt | head

How many lines contain "river"?  We can count by piping into the word count tool `wc`.

In [29]:
!grep river siddhartha.txt | wc

    109    1365    7443


That's 109 matching lines, containing 1365 words and 7979 characters.  If you just wanted the lines by themselves, use `wc -l`:

In [30]:
!grep river siddhartha.txt | wc -l

109


What if we want to match both upper- and lower-case text?  Use `grep -i`:

In [31]:
!grep time siddhartha.txt | wc -l

166


In [32]:
!grep -i time siddhartha.txt | wc -l

167


### Your turn

How many lines in *Siddhartha* contain "other" (just lower-case)?  Start by using `grep` to extract lines that match the word "other" in `siddhartha.txt` and redirecting it to a file called `other-lines.txt`.

In [33]:
!grep other siddhartha.txt > other-lines.txt 

In [34]:
%sc h_other = head -1 other-lines.txt
assert "other" in h_other

In [35]:
%sc t_other = tail -1 other-lines.txt
assert "other" in t_other

Now count up the lines in the file you created using `wc`.

In [45]:
line_count = !wc -l < other-lines.txt
print(line_count)

['127']


In [46]:
assert "127" in line_count

Your answer should be 127!

### Counting words with `grep`

By piping commands together we can do a lot of powerful things right at the command line.  Let's create a count of the most commonly occurring words in *Siddhartha*.  To do that, we could write a Python script that just counts words, but with the command line shell tools we only need to put a proper pipeline together and we can often accomplish tasks like this in one line.

First we need to split up the text lines into a word per line.  There are `grep` flags for that!

In [48]:
!cat siddhartha.txt | grep -oE '\w{{2,}}' | head -10

The
Project
Gutenberg
EBook
of
Siddhartha
by
Herman
Hesse
This
grep: write error: Broken pipe
cat: write error: Broken pipe


Now we need to sort them and count the unique tokens.  `sort` solves the first problem.

In [49]:
!cat siddhartha.txt | grep -oE '\w{{2,}}' | sort | head -10

000
1500
1887
20
2001
2008
2011
2013
23
2500
sort: write failed: 'standard output': Broken pipe
sort: write error


And `uniq -c` solves the second problem.

In [50]:
!cat siddhartha.txt | grep -oE '\w{{2,}}' | sort | uniq -c | head -25

      1 000
      1 1500
      1 1887
      1 20
      1 2001
      1 2008
      1 2011
      1 2013
      1 23
      4 2500
      1 30
      1 4557
      1 50
      2 501
      1 596
      1 60
      1 6221541
      1 64
      1 801
      1 809
      1 84116
      2 90
      1 99712
      2 abandon
      2 abandoned
uniq: write error: Broken pipe


But there's a catch... do you see it?

We need to convert all the words down into lower case so that we are correctly counting unique words.  There's another command, `tr`, for that.

In [51]:
!cat siddhartha.txt | grep -oE '\w{{2,}}' | tr '[:upper:]' '[:lower:]' | sort | uniq -c | head -25

      1 000
      1 1500
      1 1887
      1 20
      1 2001
      1 2008
      1 2011
      1 2013
      1 23
      4 2500
      1 30
      1 4557
      1 50
      2 501
      1 596
      1 60
      1 6221541
      1 64
      1 801
      1 809
      1 84116
      2 90
      1 99712
      2 abandon
      2 abandoned
uniq: write error: Broken pipe


...and if we want to know only the top 10 words in Siddhartha, we need to sort the output.

In [52]:
!cat siddhartha.txt | grep -oE '\w{{2,}}' | tr '[:upper:]' '[:lower:]' | sort | uniq -c | sort | head -10

      1 000
    101 do
    106 more
    108 any
    108 so
     10 access
     10 agree
     10 arms
     10 ask
     10 ate
sort: write failed: 'standard output': Broken pipe
sort: write error


But that sorts by character, not number.  Fortunately, `sort -n` does what we want.

In [53]:
!cat siddhartha.txt | grep -oE '\w{{2,}}' | tr '[:upper:]' '[:lower:]' | sort | uniq -c | sort -n | head -10

      1 000
      1 1500
      1 1887
      1 20
      1 2001
      1 2008
      1 2011
      1 2013
      1 23
      1 30
sort: write failed: 'standard output': Broken pipe
sort: write error


But that's the wrong end of the list!  Two ways to fix that:  (a) use `tail` instead of `head`; (b) use `sort -rn`, which will sort in reverse order.  Let's try the latter.

In [54]:
!cat siddhartha.txt | grep -oE '\w{{2,}}' | tr '[:upper:]' '[:lower:]' | sort | uniq -c | sort -rn | head -10

   2221 the
   1434 and
   1225 to
   1106 of
    960 he
    708 his
    686 in
    540 you
    524 had
    512 was
sort: write failed: 'standard output': Broken pipe
sort: write error


### Your turn

Download *Alice in Wonderland* from http://www.gutenberg.org/cache/epub/11/pg11.txt (or https://github.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/raw/master/exercises/pg11.txt if the first url doesn't work).

In [55]:
!wget http://www.gutenberg.org/cache/epub/11/pg11.txt

--2016-09-09 09:56:21--  http://www.gutenberg.org/cache/epub/11/pg11.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167518 (164K) [text/plain]
Saving to: ‘pg11.txt’

pg11.txt            100%[===================>] 163.59K   995KB/s    in 0.2s    

2016-09-09 09:56:21 (995 KB/s) - ‘pg11.txt’ saved [167518/167518]



In [57]:
assert 'pg11.txt' in os.listdir('.')

Now rename `pg11.txt` to `alice.txt`.

In [59]:
!mv pg11.txt alice.txt

mv: cannot stat 'pg11.txt': No such file or directory


In [61]:
assert 'alice.txt' in os.listdir('.')

Take a look at the next cell.  Will it find the top 25 unique words in *Alice in Wonderland* successfully?

In [62]:
!cat alice.txt | grep -oE '\w{{2,}}' | sort | uniq -c | head -25

      1 000
      4 11
      1 1500
      1 1887
      1 1994
      2 20
      1 2001
      1 2008
      1 2011
      1 25
      1 30
      1 4557
      1 50
      2 501
      1 596
      1 60
      1 6221541
      1 64
      1 801
      1 809
      1 84116
      2 90
      1 99712
      2 abide
      1 able
uniq: write error: Broken pipe


Describe what needs to be done to the previous cell to get it to work correctly.  Describe it using words, explaining the issues, rather than using shell commands!
No,it didn't find the top 25 unique words successfully. 1.It didn't consider small and upper case letter. We need to use "tr '[:upper:]' '[:lower:]'" to convert all the upper cases into lower case. 2.As the command sorts the words in alphabetical order, we need to use sort -c for character and sort -n for number. 3.We need to use sort -rn to sort in reverse order so that the top 25 can be selected.

In [ ]:
#No,it didn't find the top 25 unique words successfully. 
#1.It didn't consider small and upper case letter. We need to use "tr '[:upper:]' '[:lower:]'" to convert all the upper cases into lower case. 
#2.As the command sorts the words in alphabetical order, we need to use sort -c for character and sort -n for number. 
#3.We need to use sort -rn to sort in reverse order so that the top 25 can be selected, or use tail -25 rather than head.

Okay, now implement your solution using shell commands with a pipeline.

In [64]:
!cat alice.txt | grep -oE '\w{{2,}}' | tr '[:upper:]' '[:lower:]' | sort | uniq -c | sort -rn | head -25

   1818 the
    940 and
    809 to
    631 of
    610 it
    553 she
    481 you
    462 said
    431 in
    403 alice
    358 was
    330 that
    274 as
    248 her
    228 with
    227 at
    204 on
    200 all
    181 this
    179 for
    178 had
    175 but
    167 be
    166 not
    155 they
sort: write failed: 'standard output': Broken pipe
sort: write error


## Part 2 - csvkit basics

Let's look at some CSV data using csvkit.  Download the 2015 fourth quarter trip dataset from [Capital Bikeshare's trip history data](https://www.capitalbikeshare.com/trip-history-data):

In [65]:
!wget https://s3.amazonaws.com/capitalbikeshare-data/2015-Q4-cabi-trip-history-data.zip

--2016-09-09 10:11:25--  https://s3.amazonaws.com/capitalbikeshare-data/2015-Q4-cabi-trip-history-data.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.65.243
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.65.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14308794 (14M) [application/zip]
Saving to: ‘2015-Q4-cabi-trip-history-data.zip’

2015-Q4-cabi-trip-h 100%[===================>]  13.65M  5.93MB/s    in 2.3s    

2016-09-09 10:11:28 (5.93 MB/s) - ‘2015-Q4-cabi-trip-history-data.zip’ saved [14308794/14308794]



Let's unzip it, rename it to something short, and take a look.

In [66]:
!unzip 2015-Q4-cabi-trip-history-data.zip

Archive:  2015-Q4-cabi-trip-history-data.zip
  inflating: 2015-Q4-Trips-History-Data.csv  


In [67]:
!mv 2015-Q4-Trips-History-Data.csv q4.csv

In [68]:
!head q4.csv

csvkit gives us great tools for examining and working with CSV data.  We start by looking at the columns:

In [70]:
!csvcut -n q4.csv

  1: Duration (ms)
  2: Start date
  3: End date
  4: Start station number
  5: Start station
  6: End station number
  7: End station
  8: Bike #
  9: Member type


We can also extract just a few columns with `csvcut`:

In [71]:
!csvcut -c1,5,7 q4.csv | head -10

Duration (ms),Start station,End station
166050,Park Rd & Holmead Pl NW,14th & Harvard St NW
379172,34th & Water St NW,25th St & Pennsylvania Ave NW
696038,17th & Corcoran St NW,17th & Corcoran St NW
219423,Adams Mill & Columbia Rd NW,Calvert St & Woodley Pl NW
253230,11th & Kenyon St NW,11th & Kenyon St NW
655251,18th St & Pennsylvania Ave NW,18th St & Wyoming Ave NW
309212,11th & S St NW,18th & R St NW
776195,34th St & Wisconsin Ave NW,17th & Corcoran St NW
151604,Wilson Blvd & N Uhle St,N Veitch  & 20th St N


...and make it look better with `csvlook`:

In [72]:
!csvcut -c1,5,7 q4.csv | head -10 | csvlook

|----------------+-------------------------------+--------------------------------|
|  Duration (ms) | Start station                 | End station                    |
|----------------+-------------------------------+--------------------------------|
|  166050        | Park Rd & Holmead Pl NW       | 14th & Harvard St NW           |
|  379172        | 34th & Water St NW            | 25th St & Pennsylvania Ave NW  |
|  696038        | 17th & Corcoran St NW         | 17th & Corcoran St NW          |
|  219423        | Adams Mill & Columbia Rd NW   | Calvert St & Woodley Pl NW     |
|  253230        | 11th & Kenyon St NW           | 11th & Kenyon St NW            |
|  655251        | 18th St & Pennsylvania Ave NW | 18th St & Wyoming Ave NW       |
|  309212        | 11th & S St NW                | 18th & R St NW                 |
|  776195        | 34th St & Wisconsin Ave NW    | 17th & Corcoran St NW          |
|  151604        | Wilson Blvd & N Uhle St       | N Veitch  & 20th St N    

It gets even better.  Try `csvgrep`:

In [73]:
!csvcut -c1,5,7 q4.csv | csvgrep -c3 -m '21st & I St NW' | head -10 | csvlook

|----------------+----------------------------------+-----------------|
|  Duration (ms) | Start station                    | End station     |
|----------------+----------------------------------+-----------------|
|  872420        | 5th St & Massachusetts Ave NW    | 21st & I St NW  |
|  1249207       | 11th & Kenyon St NW              | 21st & I St NW  |
|  548066        | 17th & Corcoran St NW            | 21st & I St NW  |
|  604362        | 18th St & Wyoming Ave NW         | 21st & I St NW  |
|  1556978       | Army Navy Dr & S Nash St         | 21st & I St NW  |
|  586738        | New Hampshire Ave & T St NW      | 21st & I St NW  |
|  259421        | 17th & K St NW / Farragut Square | 21st & I St NW  |
|  269179        | M St & Pennsylvania Ave NW       | 21st & I St NW  |
|  227962        | 25th St & Pennsylvania Ave NW    | 21st & I St NW  |
|----------------+----------------------------------+-----------------|


But wait, there's more:

In [74]:
!csvcut -c1,5,7 q4.csv | csvgrep -c3 -m '21st & I St NW' | csvsort -c2 | head -10 | csvlook

|----------------+-----------------------+-----------------|
|  Duration (ms) | Start station         | End station     |
|----------------+-----------------------+-----------------|
|  890758        | 10th & E St NW        | 21st & I St NW  |
|  1074547       | 10th & E St NW        | 21st & I St NW  |
|  751900        | 10th & E St NW        | 21st & I St NW  |
|  938747        | 10th & Florida Ave NW | 21st & I St NW  |
|  1001635       | 10th & Florida Ave NW | 21st & I St NW  |
|  837341        | 10th & Florida Ave NW | 21st & I St NW  |
|  891224        | 10th & Florida Ave NW | 21st & I St NW  |
|  1070924       | 10th & Florida Ave NW | 21st & I St NW  |
|  799070        | 10th & Florida Ave NW | 21st & I St NW  |
|----------------+-----------------------+-----------------|


And you can perform basic statistics very easily:

In [75]:
!csvcut -c1,5,7 q4.csv | csvgrep -c3 -m '21st & I St NW' | csvcut -c1 | csvstat

  1. Duration (ms)
	<class 'int'>
	Nulls: False
	Min: 65972
	Max: 29254066
	Sum: 3721091088
	Mean: 867387.2
	Median: 678586.5
	Standard Deviation: 1148902.3949097062
	Unique values: 4281
	5 most frequent values:
		417735:	2
		134297:	2
		586081:	2
		466109:	2
		915332:	2

Row count: 4290


### Your turn

Which set of trips had the longer average trip duration:  trips *starting* at "Massachusetts Ave & Dupont Circle NW", or trips *ending* at "Massachusetts Ave & Dupont Circle NW"?

Use as many new cells as you need to compute the answer, and then write in your answer below in the "YOUR ANSWER HERE" cell.

In [78]:
# perform basic statistics for the trips starting at "Massachusetts Ave & Dupont Circle NW"
!csvcut -c1,5,7 q4.csv | csvgrep -c2 -m 'Massachusetts Ave & Dupont Circle NW' | csvcut -c1 | csvstat

  1. Duration (ms)
	<class 'int'>
	Nulls: False
	Min: 61260
	Max: 74022396
	Sum: 12050906879
	Mean: 946431.0750804995
	Median: 607205
	Standard Deviation: 1910753.059343867
	Unique values: 12672
	5 most frequent values:
		257138:	3
		454643:	2
		387667:	2
		374726:	2
		1150975:	2

Row count: 12733


In [79]:
# perform basic statistics for the trips ending at "Massachusetts Ave & Dupont Circle NW"
!csvcut -c1,5,7 q4.csv | csvgrep -c3 -m 'Massachusetts Ave & Dupont Circle NW' | csvcut -c1 | csvstat

  1. Duration (ms)
	<class 'int'>
	Nulls: False
	Min: 60476
	Max: 74676819
	Sum: 12098780287
	Mean: 840543.3018618869
	Median: 569422.0
	Standard Deviation: 1767359.3072143388
	Unique values: 14309
	5 most frequent values:
		497028:	2
		996079:	2
		263388:	2
		384112:	2
		451116:	2

Row count: 14394


In [ ]:
# YOUR ANSWER HERE
# Trips starting at "Massachusetts Ave & Dupont Circle NW" have the longer average trip duration (946431.0750804995 ms) 
# than trips ending at "Massachusetts Ave & Dupont Circle NW" (840543.3018618869 ms).